In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import sys,os

sys.path.append(os.path.abspath('./src'))
from webscraping_tools import *
from sql_tools import *
from tools import *

In [2]:
url = "https://stats.nba.com/stats/teamgamelogs"

request_headers = 'Accept|*/*&Accept-Encoding|gzip,deflate,br&Accept-Language|en-US,en;q=0.9&Connection|keep-alive&Host|stats.nba.com&Origin|https://www.nba.com&Referer|https://www.nba.com/&Sec-Fetch-Dest|empty&Sec-Fetch-Mode|cors&Sec-Fetch-Site|same-site&User-Agent|Mozilla/5.0(Macintosh;IntelMacOSX10_15_7) AppleWebKit/537.36(KHTML,like Gecko)Chrome/113.0.0.0 Safari/537.36&sec-ch-ua|"GoogleChrome";v="113","Chromium";v="113","Not-A.Brand";v="24"&sec-ch-ua-mobile|?0&sec-ch-ua-platform|"macOS"'
request_headers = clean_api_params(request_headers,"|","&")

In [3]:
seasons = season_range(2017,2022)

for season in tqdm.tqdm(seasons):

    measure_type = ["Base","Advanced"]

    params = query_string_parameters(measure_type[0],season)
    params = clean_api_params(params,"=")

    response = requests.get(url, headers=request_headers, params=params).json()
    base_frame = pd.DataFrame(response['resultSets'][0]['rowSet'],columns = response['resultSets'][0]['headers'])

    params = query_string_parameters(measure_type[1],season)
    params = clean_api_params(params,"=")

    response = requests.get(url, headers=request_headers, params=params).json()
    adv_frame = pd.DataFrame(response['resultSets'][0]['rowSet'],columns = response['resultSets'][0]['headers'])

    frame = base_frame.join(adv_frame.drop(columns=['SEASON_YEAR','TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP','WL','MIN','GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK']))

    frame[["TEAM_ABBREVIATION","TEAM_NAME","MATCHUP"]] = frame.apply(lambda row: pd.Series(matchup_reformat(row["MATCHUP"])), axis=1) 
    
    frame = frame.rename(columns={"TEAM_ABBREVIATION":"teamAbbr","TEAM_NAME":"opptAbbr","MATCHUP":"location","GAME_DATE":"game_date"})
    
    frame['game_date'] = pd.to_datetime(frame['game_date'])
    #frame = frame.sort_values(by=["GAME_ID","location"]).reset_index(drop=True)
    
    update_database("database.db","boxscore",frame)

  0%|          | 0/5 [00:00<?, ?it/s]

Table boxscore found in database database.db. Updating table...


 20%|██        | 1/5 [00:00<00:02,  1.40it/s]

Table boxscore found in database database.db. Updating table...


 40%|████      | 2/5 [00:01<00:02,  1.28it/s]

...

 60%|██████    | 3/5 [00:03<00:02,  1.30s/it]

Table boxscore found in database database.db. Updating table...


 80%|████████  | 4/5 [00:11<00:04,  4.03s/it]

Table boxscore found in database database.db. Updating table...


100%|██████████| 5/5 [00:21<00:00,  4.20s/it]


In [4]:
read_database("database.db","boxscore")

,SEASON_YEAR,TEAM_ID,teamAbbr,opptAbbr,GAME_ID,game_date,location,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2017-18,1610612738,BOS,CLE,0021700001,2017-10-17 00:00:00,away,L,48.0,36,...,689,46,2315,2017,2074,1830,101,68,581,1103
1,2017-18,1610612739,CLE,BOS,0021700001,2017-10-17 00:00:00,home,W,48.0,38,...,2118,1859,1681,804,830,1919,1720,1441,1063,1307
2,2017-18,1610612745,HOU,GSW,0021700002,2017-10-17 00:00:00,away,W,48.0,47,...,553,677,381,142,383,861,648,879,581,1358
3,2017-18,1610612744,GSW,HOU,0021700002,2017-10-17 00:00:00,home,L,48.0,43,...,689,1001,1649,762,1625,682,2101,2037,1063,1154
4,2017-18,1610612766,CHA,DET,0021700003,2017-10-18 00:00:00,away,L,48.0,29,...,1306,2106,102,2147,1046,730,1906,2098,1547,805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11653,2021-22,1610612745,HOU,MIN,0022100008,2021-10-20 00:00:00,away,L,48.0,40,...,2397,2080,2041,1659,1182,2368,1964,1766,79,31
11654,2021-22,1610612747,LAL,GSW,0022100002,2021-10-19 00:00:00,home,L,48.0,45,...,155,1380,944,216,488,97,1117,1259,537,334
11655,2021-22,1610612751,BKN,MIL,0022100001,2021-10-19 00:00:00,away,L,48.0,37,...,2203,2110,2153,844,1685,1827,912,1434,3,2026
11656,2021-22,1610612744,GSW,LAL,0022100002,2021-10-19 00:00:00,away,W,48.0,41,...,1797,2067,2231,1511,1973,973,1065,1418,537,2127


# Odds Data

In [385]:
import requests

def season_format(season):
    return str(season)+'-'+str(season+1)[-2:]

url = "https://www.rotowire.com/betting/nba/tables/games-archive.php"
odds_page = requests.get(url).json()
odds_data = pd.DataFrame(odds_page,columns = odds_page[0].keys())

odds_data['game_date'] = pd.to_datetime(odds_data['game_date'])
odds_data = odds_data[['season','game_date','home_team_abbrev','visit_team_abbrev','game_over_under','spread']]
odds_data['season'] = odds_data['season'].apply(lambda x: season_format(int(x)))

odds_data = odds_data.rename(columns={"home_team_abbrev":"teamAbbr","visit_team_abbrev":"opptAbbr","game_over_under":"Open_Line_OU"})

odds_data

,season,game_date,teamAbbr,opptAbbr,Open_Line_OU,spread
0,2017-18,2017-10-17,CLE,BOS,216.0,4.5
1,2017-18,2017-10-17,GSW,HOU,231.0,9.5
2,2017-18,2017-10-18,DET,CHA,202.0,2.5
3,2017-18,2017-10-18,IND,BKN,216.5,3.0
4,2017-18,2017-10-18,ORL,MIA,205.5,3.5
...,...,...,...,...,...,...
7522,2022-23,2023-05-11,PHX,DEN,226.0,1.5
7523,2022-23,2023-05-12,MIA,NYK,204.5,6.5
7524,2022-23,2023-05-12,LAL,GSW,218.0,3.5
7525,2022-23,2023-05-14,BOS,PHI,201.5,6.0


In [386]:
def update_database(db_file,table,new_frame):
    """
    Updates table in db_file with data from new_frame. Only appends rows if it is not already in the database.
    
    :db_file: (str) database file name
    :table: (str) SQL table name
    :new_frame: (pd.DataFrame) frame containing new data
    """
    conn = connect_database(db_file)

    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}'"
    result = pd.read_sql_query(query, conn).shape[0]>0
    
    if not result:
        print(f"Table {table} not found in database {db_file}. Creating new table...")
        new_frame.to_sql(table, conn, if_exists='replace',index=False)
        conn.commit()
        conn.close()
        return
    
    print(f"Table {table} found in database {db_file}. Updating table...")
    new_frame.to_sql('temp_table', conn, if_exists='replace',index=False)
    
    conn.execute(f'''
        INSERT INTO {table} 
        SELECT * FROM temp_table 
        WHERE NOT EXISTS (
            SELECT * FROM {table} 
            WHERE {table}.GAME_ID = temp_table.GAME_ID 
            AND {table}.TEAM_ABBREVIATION = temp_table.TEAM_ABBREVIATION
        )
    ''')

    conn.execute('''
        DROP TABLE temp_table;
    ''')

    conn.commit()
    conn.close()

In [387]:
update_database("database.db","odds",odds_data)

Table odds not found in database database.db. Creating new table...
